In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [7]:
from sklearn.metrics import accuracy_score, log_loss
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import r2_score,mean_squared_error

In [14]:
from sklearn.neighbors import KNeighborsRegressor

In [5]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.preprocessing import StandardScaler

In [6]:
import os
os.chdir("/Users/darshmac/Documents/cdac/For DBDA/Advance Analytics/Datasets")

In [8]:
boston = pd.read_csv("Boston.csv")
boston

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273,21.0,393.45,6.48,22.0


In [9]:
X = boston.drop('medv',axis=1).values
y = boston['medv'].values

In [11]:
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.3,random_state=23)

In [12]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(354, 13)
(354,)
(152, 13)
(152,)


In [15]:
knn = KNeighborsRegressor(n_neighbors =3)
knn.fit(X_train,y_train)

KNeighborsRegressor(n_neighbors=3)

In [17]:
y_pred = knn.predict(X_test)

In [18]:
print(r2_score(y_test,y_pred))

0.42391065570620046


In [19]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [20]:
std_scaler = StandardScaler()

In [22]:
from sklearn.pipeline import Pipeline

In [23]:
std_pipe = Pipeline([('stdsc',std_scaler),('knn',knn)])

In [24]:
std_pipe.fit(X_train,y_train)

Pipeline(steps=[('stdsc', StandardScaler()),
                ('knn', KNeighborsRegressor(n_neighbors=3))])

In [25]:
y_pred = std_pipe.predict(X_test)

In [28]:
print(r2_score(y_test,y_pred))#significant increase in accuracy after scaling

0.8076398606818813


In [29]:
mx_scaler = MinMaxScaler()

In [30]:
mx_pipline = Pipeline([('mxscl',mx_scaler),('knn',knn)])
mx_pipline.fit(X_train,y_train)

Pipeline(steps=[('mxscl', MinMaxScaler()),
                ('knn', KNeighborsRegressor(n_neighbors=3))])

In [31]:
y_pred = mx_pipline.predict(X_test)

In [32]:
print(r2_score(y_test,y_pred))

0.8088868627293495


In [33]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

In [35]:
knn = KNeighborsRegressor(n_neighbors =3)
knn.fit(X_train,y_train)

KNeighborsRegressor(n_neighbors=3)

In [36]:
kfold = KFold(n_splits=5,shuffle=True,random_state=23)

In [37]:
print(knn.get_params())

{'algorithm': 'auto', 'leaf_size': 30, 'metric': 'minkowski', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 3, 'p': 2, 'weights': 'uniform'}


In [38]:
params = {'n_neighbors':np.arange(1,31)}

In [39]:
gcv = GridSearchCV(knn,param_grid=params,cv = kfold)

In [40]:
gcv.fit(X,y)

GridSearchCV(cv=KFold(n_splits=5, random_state=23, shuffle=True),
             estimator=KNeighborsRegressor(n_neighbors=3),
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30])})

In [42]:
print(gcv.best_params_)
print(gcv.best_score_)

{'n_neighbors': 3}
0.544002500199357


In [43]:
#using pipeline
std_pipe = Pipeline([('stdsc',std_scaler),('knn',knn)])

In [44]:
print(std_pipe.get_params())

{'memory': None, 'steps': [('stdsc', StandardScaler()), ('knn', KNeighborsRegressor(n_neighbors=3))], 'verbose': False, 'stdsc': StandardScaler(), 'knn': KNeighborsRegressor(n_neighbors=3), 'stdsc__copy': True, 'stdsc__with_mean': True, 'stdsc__with_std': True, 'knn__algorithm': 'auto', 'knn__leaf_size': 30, 'knn__metric': 'minkowski', 'knn__metric_params': None, 'knn__n_jobs': None, 'knn__n_neighbors': 3, 'knn__p': 2, 'knn__weights': 'uniform'}


In [48]:
params = {'knn__n_neighbors':np.arange(1,31)}

In [49]:
gcv = GridSearchCV(std_pipe,param_grid=params,cv=kfold)

In [50]:
gcv.fit(X,y)

GridSearchCV(cv=KFold(n_splits=5, random_state=23, shuffle=True),
             estimator=Pipeline(steps=[('stdsc', StandardScaler()),
                                       ('knn',
                                        KNeighborsRegressor(n_neighbors=3))]),
             param_grid={'knn__n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30])})

In [51]:
print(gcv.best_params_)
print(gcv.best_score_)

{'knn__n_neighbors': 2}
0.7793947733827284


In [52]:
mx_pipline = Pipeline([('mxscl',mx_scaler),('knn',knn)])

In [53]:
print(mx_pipline.get_params())

{'memory': None, 'steps': [('mxscl', MinMaxScaler()), ('knn', KNeighborsRegressor(n_neighbors=3))], 'verbose': False, 'mxscl': MinMaxScaler(), 'knn': KNeighborsRegressor(n_neighbors=3), 'mxscl__clip': False, 'mxscl__copy': True, 'mxscl__feature_range': (0, 1), 'knn__algorithm': 'auto', 'knn__leaf_size': 30, 'knn__metric': 'minkowski', 'knn__metric_params': None, 'knn__n_jobs': None, 'knn__n_neighbors': 3, 'knn__p': 2, 'knn__weights': 'uniform'}


In [54]:
params = {'knn__n_neighbors':np.arange(1,31)}

In [55]:
gcv = GridSearchCV(mx_pipline,param_grid=params,cv=kfold)

In [56]:
gcv.fit(X,y)

GridSearchCV(cv=KFold(n_splits=5, random_state=23, shuffle=True),
             estimator=Pipeline(steps=[('mxscl', MinMaxScaler()),
                                       ('knn',
                                        KNeighborsRegressor(n_neighbors=3))]),
             param_grid={'knn__n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30])})

In [57]:
print(gcv.best_params_)
print(gcv.best_score_)

{'knn__n_neighbors': 2}
0.7716352177204511
